In [1]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, random

In [2]:
million=1000000
nbMillions=1
nSamples=50
lexiquePrefix="MGC-160104"
tiragePrefix="MGC-170504"
echantillon="-%dMo"%nbMillions

In [5]:
with open("/Users/gilles/Box Sync/2015-Data/MGC-Vienna/"+lexiquePrefix+'-Verbes2.pkl', 'rb') as input:
    lexiqueBase = pickle.load(input)

In [6]:
lexiqueBase["freqcum"]= (lexiqueBase["freq"].cumsum()*1000).astype(int)

In [7]:
rangeTop=lexiqueBase["freqcum"].max()

In [8]:
del lexiqueBase["ext"]
del lexiqueBase["cs"]
del lexiqueBase["ms"]
del lexiqueBase["vs"]
del lexiqueBase["prob"]

In [9]:
lexiqueBase["tir1"]=0

In [10]:
def tirageIndex():
    tirage=[]
    for n in range(nbMillions*million):
        tirage.append(random.randrange(rangeTop))
    return tirage

In [11]:
%%time
def tirage2triage(tirage):    
    triage=sorted(tirage)
    freqTop=0
    indexMin=0
    tirs={}
    lexique=lexiqueBase.copy()
    lexique["tir1"]=0
    for num,tir in enumerate(triage[:]):
        if tir > freqTop:
            indexMin=lexique[lexique["freqcum"]>=tir][0:1].index.astype(int)[0]
            freqTop=lexique.ix[indexMin,'freqcum']
            tirs[indexMin]=0
        tirs[indexMin] += 1
        if num%5000000==0:
            print num,
    print
    for indexNum in tirs:
        lexique.ix[indexNum,'tir1']+=tirs[indexNum]
    return lexique

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 6.91 µs


In [12]:
for nSample in range(nSamples):
    tirage=tirageIndex()
    %time lexique=tirage2triage(tirage)
    print
    print "nSample",nSample
    print lexique[lexique["tir1"]!=0]["tir1"].sum()
    print lexique[lexique["tir1"]!=0]["tir1"].count()
    with open("/Users/gilles/Box Sync/2015-Data/"+tiragePrefix+"-%2d"%nSample+echantillon+'-Tirage.pkl', 'wb') as output:
        pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
#    lexiqueBase["tir1"]+=lexique["tir1"]
#print lexiqueBase[lexiqueBase["tir1"]!=0]["tir1"].sum()
#print lexiqueBase[lexiqueBase["tir1"]!=0]["tir1"].count()
#with open("/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+tiragePrefix+"-%2d"%nSamples+"-Total"+echantillon+'-Tirage.pkl', 'wb') as output:
#    pickle.dump(lexiqueBase, output, pickle.HIGHEST_PROTOCOL)


0
CPU times: user 15min 1s, sys: 2min 36s, total: 17min 38s
Wall time: 17min 53s

nSample 0
1000000
43321
0
CPU times: user 17min 5s, sys: 3min, total: 20min 6s
Wall time: 21min 18s

nSample 1
1000000
43272
0
CPU times: user 14min 37s, sys: 2min 2s, total: 16min 40s
Wall time: 16min 43s

nSample 2
1000000
43292
0
CPU times: user 15min 58s, sys: 2min 43s, total: 18min 42s
Wall time: 18min 52s

nSample 3
1000000
43168
0
CPU times: user 16min 26s, sys: 3min 3s, total: 19min 30s
Wall time: 19min 52s

nSample 4
1000000
43310
0
CPU times: user 15min 41s, sys: 2min 49s, total: 18min 31s
Wall time: 2h 39min 31s

nSample 5
1000000
43296
0
CPU times: user 18min 24s, sys: 3min 9s, total: 21min 34s
Wall time: 23min 15s

nSample 6
1000000
43294
0
CPU times: user 17min 10s, sys: 2min 14s, total: 19min 25s
Wall time: 19h 18min 52s

nSample 7
1000000
43496
0
CPU times: user 15min 38s, sys: 2min 23s, total: 18min 1s
Wall time: 19min 6s

nSample 8
1000000
43182
0
CPU times: user 15min 59s, sys: 2min 47s